In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter
import seaborn as sns
from sklearn.model_selection import cross_val_score

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [27]:
train=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\janta\train.csv')
test=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\janta\test.csv')
sub=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\janta\sample_submission.csv')

In [29]:
train.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1
3,F00000005,257,1,1,1,0,0.0,0,2,1
4,F00000006,342,1,0,1,0,0.0,0,2,1


In [30]:
y = train.pop('Crop_Damage')
X = train
test_id = test.pop('ID')

In [31]:
X=X.drop('ID',axis=1)

In [32]:
import random
nans = X['Number_Weeks_Used'].isna()
length = sum(nans)
length
replacement = random.choices([10,15,20,22,25,30,35,40,45,50], weights=[.1,.1,.1,.1,.1,.1,.1,.1,.1,.1], k=length)
X.loc[nans,'Number_Weeks_Used'] = replacement

In [33]:
nant = test['Number_Weeks_Used'].isna()
length1 = sum(nant)
length1
replacement1 = random.choices([10,15,20,22,25,30,35,40,45,50], weights=[.1,.1,.1,.1,.1,.1,.1,.1,.1,.1], k=length1)
test.loc[nant,'Number_Weeks_Used'] = replacement1

In [57]:
# cat_features=[2,3,4,6]
# final_model = CatBoostClassifier(
#         iterations=1500,logging_level="Silent", random_seed=0,
#         eval_metric="AUC")
# final_model.fit(X,y, cat_features=cat_features )

In [58]:
# sum(cross_val_score(final_model, X, y, cv=5))/5

In [34]:
catboost = CatBoostClassifier(iterations=2000, learning_rate=0.003,metrics='auc',verbose=500)
catboost.fit(X, y)

0:	learn: 1.0617418	total: 144ms	remaining: 4m 47s
500:	learn: 0.4171930	total: 31.7s	remaining: 1m 34s
1000:	learn: 0.4103808	total: 59.8s	remaining: 59.7s
1500:	learn: 0.4057883	total: 1m 28s	remaining: 29.3s
1999:	learn: 0.4018535	total: 1m 54s	remaining: 0us


In [35]:
sum(cross_val_score(catboost, X, y, cv=5))/5

0:	learn: 1.0618045	total: 111ms	remaining: 3m 41s
500:	learn: 0.4176660	total: 23.6s	remaining: 1m 10s
1000:	learn: 0.4101215	total: 44.8s	remaining: 44.7s
1500:	learn: 0.4046069	total: 1m 8s	remaining: 22.8s
1999:	learn: 0.3999731	total: 1m 31s	remaining: 0us
0:	learn: 1.0617699	total: 82.6ms	remaining: 2m 45s
500:	learn: 0.4160890	total: 25.7s	remaining: 1m 16s
1000:	learn: 0.4084662	total: 47.8s	remaining: 47.7s
1500:	learn: 0.4029323	total: 1m 9s	remaining: 23.1s
1999:	learn: 0.3982547	total: 1m 34s	remaining: 0us
0:	learn: 1.0617748	total: 53.8ms	remaining: 1m 47s
500:	learn: 0.4167975	total: 28s	remaining: 1m 23s
1000:	learn: 0.4089400	total: 52.7s	remaining: 52.6s
1500:	learn: 0.4036203	total: 1m 14s	remaining: 24.9s
1999:	learn: 0.3989657	total: 1m 36s	remaining: 0us
0:	learn: 1.0617276	total: 122ms	remaining: 4m 4s
500:	learn: 0.4162898	total: 26.4s	remaining: 1m 18s
1000:	learn: 0.4083898	total: 47.9s	remaining: 47.8s
1500:	learn: 0.4029176	total: 1m 9s	remaining: 23.1s
1999

0.8456638526253798

In [41]:
xgboost = XGBClassifier()
xgboost.fit(X, y)
sum(cross_val_score(xgboost, X, y, cv=5))/5

0.8457876498230614

In [37]:
pred_c = catboost.predict_proba(test)
pred_x = xgboost.predict_proba(test)

In [38]:
final_res = []
for i in range(len(pred_x)):
    final_res.append(np.argmax((pred_x[i]*.4 + pred_c[i]*.6)/2))

pd.DataFrame({'ID': test_id, 'Crop_Damage':final_res}).to_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\janta\submissionM.csv', index=False)

In [ ]:
# sub["target"] = (sub_lgb["target"] * 0.5 + sub_xgb["target"] * 0.3 + sub_cat["target"] * 0.2)